<a href="https://colab.research.google.com/github/joao-dias-25/dataeng-spark/blob/main/spark/challenges/challenge_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHALLENGE 2
##  Implement CLEANSING process
- Set up path in the "lake"
  - !mkdir -p /content/lake/silver

- Read data from BRONZE layer as PARQUET:
    - vehicles - path: /content/lake/bronze/vehicles
    - lines - path: /content/lake/bronze/lines
    - municipalities - path: /content/lake/bronze/municipalities

- Transformations
  - vehicles
    - rename "lat" and "lon" to "latitude" and "longitude" respectively
    - remove possible duplicates
    - remove rows when the column CURRENT_STATUS is null
    - remove any corrupted record
  - lines
    - remove duplicates
    - remove rows when the column LONG_NAME is null
    - remove any corrupted record
  - municipalities
    - remove duplicates
    - remove rows when the columns NAME or DISTRICT_NAME are null
    - remove any corrupted record

- Write data as PARQUET into the SILVER layer (/content/lake/silver)
  - Partition "vehicles" by "date"(created in the ingestion)
  - Paths:
    - vehicles - path: /content/lake/silver/vehicles
    - lines - path: /content/lake/silver/lines
    - municipalities - path: /content/lake/silver/municipalities

# Setting up PySpark

In [2]:
%pip install pyspark

In [3]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('Spark Course').config('spark.ui.port', '4050').getOrCreate()
sc = spark.sparkContext

In [37]:

from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import requests

class ETLFlow:
    def __init__(self, spark: SparkSession) -> None:
        self.spark = spark

    def extract_from_file(self, format: str, path: str, **kwargs) -> DataFrame:
        df = self.spark.read.format(format).load(path)
        return df

    def extract_from_api(self, url: str, schema: StructType = None):
      response = requests.get(url)
      rdd = spark.sparkContext.parallelize(response.json())
      if schema:
        df = spark.read.schema(schema).json(rdd)
      else:
        df = spark.read.json(rdd)
      return df

    def load(self, df: DataFrame, format: str, path: str, **kwargs) -> None:
        df.write.mode("overwrite").format(format).save(path)

    def load_partition(self, df: DataFrame, partition:str, format: str, path: str, **kwargs) -> None:
        df.write.mode("overwrite").partitionBy(partition).format(format).save(path)

class ETLTask(ETLFlow):

    def __init__(self, spark: SparkSession) -> None:
        self.spark = spark

    def ingestion_vehicles(self):
      vehicle_schema = StructType([StructField('bearing', IntegerType(), True),
                                  StructField('block_id', StringType(), True),
                                  StructField('current_status', StringType(), True),
                                  StructField('id', StringType(), True),
                                  StructField('lat', FloatType(), True),
                                  StructField('line_id', StringType(), True),
                                  StructField('lon', FloatType(), True),
                                  StructField('pattern_id', StringType(), True),
                                  StructField('route_id', StringType(), True),
                                  StructField('schedule_relationship', StringType(), True),
                                  StructField('shift_id', StringType(), True),
                                  StructField('speed', FloatType(), True),
                                  StructField('stop_id', StringType(), True),
                                  StructField('timestamp', TimestampType(), True),
                                  StructField('trip_id', StringType(), True)])

      df = self.extract_from_api(url="https://api.carrismetropolitana.pt/vehicles", schema=vehicle_schema)
      df = df.withColumn('date', date_format('timestamp',"yyyyMMdd"))
      self.load(df=df, format="parquet", path="/content/lake/bronze/vehicles")

    def ingestion_lines(self):

      lines_schema = StructType([
                                 StructField('_corrupt_record', StringType(), True),
                                  StructField('color', StringType(), True),
                                  StructField("facilities",  ArrayType(StringType(), True), True),
                                  StructField('id', StringType(), True),
                                  StructField("localities",  ArrayType(StringType(), True), True),
                                  StructField('long_name', StringType(), True),
                                  StructField("municipalities",  ArrayType(StringType(), True), True),
                                  StructField("patterns",  ArrayType(StringType(), True), True),
                                  StructField("routes",  ArrayType(StringType(), True), True),
                                  StructField('short_name', StringType(), True),
                                  StructField('text_color', StringType(), True)
                                  ])
      df = self.extract_from_api(url="https://api.carrismetropolitana.pt/lines", schema=lines_schema)

      self.load(df=df, format="parquet", path="/content/lake/bronze/lines")

    def ingestion_municipalities(self):
      municipalities_schema = StructType([
                                 StructField('district_id', StringType(), True),
                                  StructField('district_name', StringType(), True),
                                  StructField("id",  StringType(), True),
                                  StructField('name', StringType(), True),
                                  StructField("prefix", StringType(), True),
                                  StructField('region_id', StringType(), True),
                                  StructField("region_name",  StringType(), True),

                                  ])

      df = self.extract_from_api(url="https://api.carrismetropolitana.pt/municipalities", schema=municipalities_schema)

      self.load(df=df, format="parquet", path="/content/lake/bronze/municipalities")

    def cleansing_vehicles(self):
      df = self.extract_from_file(format="parquet", path="/content/lake/bronze/vehicles")

      # transformations
      df = df.withColumnRenamed("lat", "latitude")
      df = df.withColumnRenamed("lon", "longitude")
      df = df.drop_duplicates()
      df = df.filter(df.current_status.isNotNull())

      self.load_partition(df=df, format="parquet", partition='date' ,path="/content/lake/silver/vehicles")

    def cleansing_lines(self):
      df = self.extract_from_file(format="parquet", path="/content/lake/bronze/lines")

      # transformations
      df = df.drop_duplicates()
      df = df.filter(df.long_name.isNotNull())
      self.load(df=df, format="parquet", path="/content/lake/silver/lines")

    def cleansing_municipalities(self):
      df = self.extract_from_file(format="parquet", path="/content/lake/bronze/municipalities")

      # transformations
      df = df.drop_duplicates()
      df = df.filter(df.name.isNotNull() & df.district_name.isNotNull())
      self.load(df=df, format="parquet", path="/content/lake/silver/municipalities")

    def enrich(self):
        pass


if __name__ == '__main__':

    # init spark
    from pyspark.sql import SparkSession
    spark = SparkSession.builder.master('local').appName('ETL Program').getOrCreate()

    print("Starting ETL program")
    etl = ETLTask(spark)

    # run tasks
    print("Running Task - Ingestion Vehicles")
    etl.ingestion_vehicles()

    print("Running Task - Ingestion lines")
    etl.ingestion_lines()

    print("Running Task - ingestion municipalities ")
    etl.ingestion_municipalities()

    print("Running Task - Cleansing Vehicles")
    etl.cleansing_vehicles()

    print("Running Task - Cleansing lines")
    etl.cleansing_lines()

    print("Running Task - Cleansing municipalities")
    etl.cleansing_municipalities()

    #etl.enrich()

    print("ETL program completed")

Starting ETL program
Running Task - Ingestion Vehicles
Running Task - Ingestion lines
Running Task - ingestion municipalities 
Running Task - Cleansing Vehicles
Running Task - Cleansing lines
Running Task - Cleansing municipalities
ETL program completed


In [28]:
spark.read.format("parquet").load("/content/lake/bronze/municipalities").show()

+-----------+-------------+----+--------------------+------+---------+----------------+
|district_id|district_name|  id|                name|prefix|region_id|     region_name|
+-----------+-------------+----+--------------------+------+---------+----------------+
|         07|        Évora|0712|        Vendas Novas|    19|    PT187|Alentejo Central|
|         11|       Lisboa|1101|            Alenquer|    20|    PT16B|           Oeste|
|         11|       Lisboa|1102|   Arruda dos Vinhos|    20|    PT16B|           Oeste|
|         11|       Lisboa|1105|             Cascais|    05|    PT170|             AML|
|         11|       Lisboa|1106|              Lisboa|    06|    PT170|             AML|
|         11|       Lisboa|1107|              Loures|    07|    PT170|             AML|
|         11|       Lisboa|1109|               Mafra|    08|    PT170|             AML|
|         11|       Lisboa|1110|              Oeiras|    12|    PT170|             AML|
|         11|       Lisboa|1111|

In [33]:
df = ETLTask(spark).extract_from_file(format="parquet", path="/content/lake/bronze/municipalities")

#remove any corrupted record
df.filter(df.name.isNotNull() & df.district_name.isNotNull()).show()

df.printSchema()





+-----------+-------------+---+----+------+---------+-----------+
|district_id|district_name| id|name|prefix|region_id|region_name|
+-----------+-------------+---+----+------+---------+-----------+
+-----------+-------------+---+----+------+---------+-----------+

root
 |-- district_id: string (nullable = true)
 |-- district_name: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- prefix: string (nullable = true)
 |-- region_id: string (nullable = true)
 |-- region_name: string (nullable = true)



In [ ]:
df = ETLTask(spark).extract_from_file(format="parquet", path="/content/lake/bronze/vehicles")

df = df.withColumnRenamed("lat", "latitude")
df = df.withColumnRenamed("lon", "longitude")
df.printSchema()
df.show()

root
 |-- bearing: integer (nullable = true)
 |-- block_id: string (nullable = true)
 |-- current_status: string (nullable = true)
 |-- id: string (nullable = true)
 |-- latitude: float (nullable = true)
 |-- line_id: string (nullable = true)
 |-- longitude: float (nullable = true)
 |-- pattern_id: string (nullable = true)
 |-- route_id: string (nullable = true)
 |-- schedule_relationship: string (nullable = true)
 |-- shift_id: string (nullable = true)
 |-- speed: float (nullable = true)
 |-- stop_id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- trip_id: string (nullable = true)
 |-- date: string (nullable = true)

+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+------+
|bearing|            block_id|current_status|      id| latitude|line_id|longitude|pattern_id|route_id|schedule_relationship|    shift_id|    sp